In [1]:
import torch
import torchvision
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch import nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import os
import numpy as np
import time
import copy 
from PIL import Image
from matplotlib import cm

In [11]:
# dataloader 설정
class CustomImageDataset(Dataset):
    def read_data_set(self):

        all_img_files = []
        all_labels = []

        class_names = os.walk(self.data_set_path).__next__()[1] 

        for index, class_name in enumerate(class_names):
            label = index
            img_dir = os.path.join(self.data_set_path, class_name)
            img_files = os.walk(img_dir).__next__()[2]

            for img_file in img_files:
                img_file = os.path.join(img_dir, img_file)
                img = Image.open(img_file)
                if img is not None:
                    all_img_files.append(img_file)
                    all_labels.append(label) # label은 0 또는 1로 저장됨 (2개의 이미지폴더가 존재하므로)

        return all_img_files, all_labels, len(all_img_files), len(class_names)

    def __init__(self, data_set_path, transforms=None):
        self.data_set_path = data_set_path
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()
        self.transforms = transforms

    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("RGB") # Tesor는 RGB이므로 변환

        if self.transforms is not None:
            image = self.transforms(image)

        return {'image': image, 'label': self.labels[index]}

    def __len__(self):
        return self.length

In [12]:
transforms_train = transforms.Compose([transforms.ToTensor(), # tensor형으로 바꾸기
                                       transforms.Resize((32, 32)),
                                       transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) # 정규화

In [13]:
train_data_set = CustomImageDataset(data_set_path='data/cat_dog/training_set',transforms=transforms_train)
train_loader = DataLoader(train_data_set, batch_size = 64, num_workers = 0, shuffle = True)

test_data_set = CustomImageDataset(data_set_path='data/cat_dog/test_set',transforms=transforms_train)
test_loader = DataLoader(test_data_set, batch_size = 64, num_workers = 0, shuffle = False)

In [14]:
# train image 크기 확인
train_data_set[0]['image'].shape

torch.Size([3, 32, 32])

In [15]:
# test image 크기 확인
test_data_set[0]['image'].shape

torch.Size([3, 32, 32])

In [16]:
# train data set 크기 확인
len(train_data_set)

8000

In [17]:
# test data set 크기 확인
len(test_data_set)

2000

In [18]:
# train data 형태 확인
type(train_data_set[0]['image'])

torch.Tensor

In [19]:
# train data 형태 확인
type(train_data_set[0]['label'])

int

In [20]:
# Model
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features = 8 * 8 * 16, out_features = 64)
        self.fc2 = nn.Linear(64,32)
        self.fc3 = nn.Linear(32,10)
    
    def forward(self, x):
        x = self.conv1(x) # input size = 32 x 32 x 3 -> 32 * 32 x 8 (가로, 세로 크기는 padding =1 을 주었으므로 불변 )
        x = F.relu(x)
        x = self.pool(x) # 32 x 32 x 8 -> 16 x 16 x 8

        x = self.conv2(x) # 16 x 16 x 8 -> 16 x 16 x 16
        x = F.relu(x)
        x = self.pool(x) # 16 x 16 x 16 -> 8 x 8 x 16
    
        # fully-connected-layer에 넣기전 다시 한번 flatten 시키기
        x = x.view(-1, 8 * 8 * 16)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.log_softmax(x)

        return x

In [21]:
# GPU 활성화
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")
print("사용 : ", DEVICE)

사용 :  cuda


In [22]:
# model 적용
model = CNN().to(DEVICE)
optimizer = optim.Adam(model.parameters(), lr=0.001)
print("사용된 모델 :", model)

사용된 모델 : CNN(
  (conv1): Conv2d(3, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=1024, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=10, bias=True)
)


In [32]:
# Train
def train(model, train_loader, optimizer):
    model.train()
    for i_batch, item in enumerate(train_loader):
        image = item['image'].to(DEVICE)
        label = item['label'].to(DEVICE)

        optimizer.zero_grad()
        output = model(image)
        loss = F.cross_entropy(output, label)
        loss.backward()
        optimizer.step()
        if i_batch % 100 == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}".format(epoch, i_batch * len(image), len(train_loader.dataset), 100. * i_batch / len(train_loader), loss.item()))

In [35]:
EPOCHS = 3
for epoch in range(1, EPOCHS + 1):
    train(model, train_loader, optimizer)
    print("[Epoch {}]".format(epoch))

ipykernel_launcher:28: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
Train Epoch: 1 [0/8000 (0%)]	Loss: 0.581202
Train Epoch: 1 [6400/8000 (80%)]	Loss: 0.500652
[Epoch 1]
Train Epoch: 2 [0/8000 (0%)]	Loss: 0.521500
Train Epoch: 2 [6400/8000 (80%)]	Loss: 0.575614
[Epoch 2]
Train Epoch: 3 [0/8000 (0%)]	Loss: 0.494878
Train Epoch: 3 [6400/8000 (80%)]	Loss: 0.620357
[Epoch 3]
